# Consistency testing

For most problems, multiple flux states can achieve the same optimum and thus we try to obtain a consistent network. By this, we mean that there will be mulitple blocked reactions in the network, which gives rise to this inconsistency. To solve this problem, we use algorithms which can detect all the blocked reactions and also give us consistent networks.

Let us take a toy network, like so:

```
v1: --> 2A
v2: A <--> B
v3: A --> D
v4: A --> C
v5: C --> D
v6: D -->
```

Here, `v{x} (x = 1, 2, 3, 4, 5, 6)` represent the flux carried by the reactions as shown above.

In [1]:
import cobra

test_model = cobra.Model("test_model")
r1 = cobra.Reaction("r1")
r2 = cobra.Reaction("r2")
r3 = cobra.Reaction("r3")
r4 = cobra.Reaction("r4")
r5 = cobra.Reaction("r5")
r6 = cobra.Reaction("r6")

test_model.add_reactions([r1, r2, r3, r4, r5, r6])

r1.reaction = "-> 2 A"
r2.reaction = "A <-> B"
r3.reaction = "A -> D"
r4.reaction = "A -> C"
r5.reaction = "C -> D"
r6.reaction = "D ->"

r1.bounds = (0.0, 3.0)
r2.bounds = (-3.0, 3.0)
r3.bounds = (0.0, 3.0)
r4.bounds = (0.0, 3.0)
r5.bounds = (0.0, 3.0)
r6.bounds = (0.0, 3.0)

test_model.objective = r6

unknown metabolite 'A' created
unknown metabolite 'B' created
unknown metabolite 'D' created
unknown metabolite 'C' created


## Using FVA

The first approach we can follow is to use FVA (Flux Variability Analysis) which among many other applications, is used to detect blocked reactions. The `cobra.flux_analysis.find_blocked_reactions()` function will return a list of all the blocked reactions obtained using FVA.

In [2]:
cobra.flux_analysis.find_blocked_reactions(test_model)

['r2']

As we see above, we are able to obtain the blocked reaction, which in this case is `r2`.

## Using FASTCC

The second approach to obtaining consistent network in `cobrapy` is to use FASTCC. Using this method, you can expect to efficiently obtain an accurate consistent network. For more details regarding the algorithm, please see [Vlassis N, Pacheco MP, Sauter T (2014)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003424).

In [3]:
consistent_model = cobra.flux_analysis.fastcc(test_model)
consistent_model.reactions

[<Reaction r1 at 0x111fac2b0>,
 <Reaction r3 at 0x111fac320>,
 <Reaction r4 at 0x111fac198>,
 <Reaction r5 at 0x111fac208>,
 <Reaction r6 at 0x111fac240>]

Similar to the FVA approach, we are able to identify that `r2` is indeed the blocked reaction.